# 📚 Introducción a Python para procesamiento de datos geoespaciales

> Colvert Gomez Rubio - Octubre 2025

# Descarga de imágenes usando [planetary-computer](https://planetarycomputer.microsoft.com/)

In [2]:
# pip install pystac-client planetary-computer

In [3]:
import pystac_client
import planetary_computer
from datetime import datetime

In [4]:
# # Conectar a Microsoft Planetary Computer (no requiere credenciales)
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [18]:
# # Buscar escenas Landsat 8-9 para Buenos Aires
search = catalog.search(
    collections=["landsat-c2-l2"],  # Landsat Collection 2 Level 2
    bbox=[-98.9, 18.8, -98.4, 19.5],  # Izta-Popo
    datetime="2025-01-01/2025-12-31",
    query={"eo:cloud_cover": {"lt": 100}}  # Máximo 100% de nubes
)

In [19]:
# Obtener todas las escenas encontradas
items = list(search.items())
items

[<Item id=LC09_L2SP_026047_20251004_02_T1>,
 <Item id=LC09_L2SP_026046_20251004_02_T1>,
 <Item id=LC09_L2SP_025047_20250927_02_T1>,
 <Item id=LC08_L2SP_026047_20250926_02_T1>,
 <Item id=LC08_L2SP_026046_20250926_02_T1>,
 <Item id=LC08_L2SP_025047_20250919_02_T1>,
 <Item id=LC09_L2SP_026047_20250918_02_T1>,
 <Item id=LC09_L2SP_026046_20250918_02_T1>,
 <Item id=LC09_L2SP_025047_20250911_02_T1>,
 <Item id=LC08_L2SP_026047_20250910_02_T1>,
 <Item id=LC08_L2SP_026046_20250910_02_T1>,
 <Item id=LC08_L2SP_025047_20250903_02_T1>,
 <Item id=LC09_L2SP_026047_20250902_02_T1>,
 <Item id=LC09_L2SP_026046_20250902_02_T1>,
 <Item id=LC09_L2SP_025047_20250826_02_T1>,
 <Item id=LC08_L2SP_026047_20250825_02_T1>,
 <Item id=LC08_L2SP_026046_20250825_02_T1>,
 <Item id=LC08_L2SP_025047_20250818_02_T1>,
 <Item id=LC09_L2SP_026047_20250817_02_T1>,
 <Item id=LC09_L2SP_026046_20250817_02_T1>,
 <Item id=LC09_L2SP_025047_20250810_02_T1>,
 <Item id=LC08_L2SP_026047_20250809_02_T1>,
 <Item id=LC08_L2SP_026046_20250

In [20]:
# Mostrar información de las primeras 5 escenas
items[:5]

[<Item id=LC09_L2SP_026047_20251004_02_T1>,
 <Item id=LC09_L2SP_026046_20251004_02_T1>,
 <Item id=LC09_L2SP_025047_20250927_02_T1>,
 <Item id=LC08_L2SP_026047_20250926_02_T1>,
 <Item id=LC08_L2SP_026046_20250926_02_T1>]

In [21]:
# primer item
item_0 = items[0]
item_0

<Item id=LC09_L2SP_026047_20251004_02_T1>

In [22]:
item_0.datetime

datetime.datetime(2025, 10, 4, 17, 0, 17, 535650, tzinfo=tzutc())

In [23]:
item_0.geometry

{'type': 'Polygon',
 'coordinates': [[[-99.78727423827206, 19.82623699153034],
   [-100.17836039079722, 18.094283444889708],
   [-98.46354463005342, 17.736505114829477],
   [-98.0579173297434, 19.471452989709537],
   [-99.78727423827206, 19.82623699153034]]]}

In [24]:
item_0.properties

{'gsd': 30,
 'created': '2025-10-08T09:26:30.550444Z',
 'sci:doi': '10.5066/P9OGBGM6',
 'datetime': '2025-10-04T17:00:17.535650Z',
 'platform': 'landsat-9',
 'proj:shape': [7761, 7601],
 'description': 'Landsat Collection 2 Level-2',
 'instruments': ['oli', 'tirs'],
 'eo:cloud_cover': 51.95,
 'proj:transform': [30.0, 0.0, 372885.0, 0.0, -30.0, 2193315.0],
 'view:off_nadir': 0,
 'landsat:wrs_row': '047',
 'landsat:scene_id': 'LC90260472025277LGN00',
 'landsat:wrs_path': '026',
 'landsat:wrs_type': '2',
 'view:sun_azimuth': 136.20319145,
 'landsat:correction': 'L2SP',
 'view:sun_elevation': 58.62064915,
 'landsat:cloud_cover_land': 51.95,
 'landsat:collection_number': '02',
 'landsat:collection_category': 'T1',
 'proj:code': 'EPSG:32614'}

In [25]:
# # Ver TODAS las bandas y assets disponibles
print(f"Todos los assets disponibles para {item_0.id}:\n")
for key, asset in item_0.assets.items():
    print(f"  - {key:20s}: {asset.title}")

Todos los assets disponibles para LC09_L2SP_026047_20251004_02_T1:

  - qa                  : Surface Temperature Quality Assessment Band
  - ang                 : Angle Coefficients File
  - red                 : Red Band
  - blue                : Blue Band
  - drad                : Downwelled Radiance Band
  - emis                : Emissivity Band
  - emsd                : Emissivity Standard Deviation Band
  - trad                : Thermal Radiance Band
  - urad                : Upwelled Radiance Band
  - atran               : Atmospheric Transmittance Band
  - cdist               : Cloud Distance Band
  - green               : Green Band
  - nir08               : Near Infrared Band 0.8
  - lwir11              : Surface Temperature Band
  - swir16              : Short-wave Infrared Band 1.6
  - swir22              : Short-wave Infrared Band 2.2
  - coastal             : Coastal/Aerosol Band
  - mtl.txt             : Product Metadata File (txt)
  - mtl.xml             : Product Metad

In [26]:
# # Ver las bandas disponibles
item = items[0]
print(f"Bandas disponibles para {item.id}:")
for key, asset in item.assets.items():
    if 'eo:bands' in asset.extra_fields:
        print(f"  - {key}: {asset.title}")

Bandas disponibles para LC09_L2SP_026047_20251004_02_T1:
  - red: Red Band
  - blue: Blue Band
  - green: Green Band
  - nir08: Near Infrared Band 0.8
  - lwir11: Surface Temperature Band
  - swir16: Short-wave Infrared Band 1.6
  - swir22: Short-wave Infrared Band 2.2
  - coastal: Coastal/Aerosol Band


In [27]:
item_0.id

'LC09_L2SP_026047_20251004_02_T1'

In [28]:
# Convertir a GeoDataFrame para tener funcionalidades geoespaciales
import geopandas as gpd
from shapely.geometry import shape

lista_escenas = []
for item in items:
    info = {"id":item.id, 'geometry': shape(item.geometry)}
    for key, value in item.properties.items():
        info[key] = value
    lista_escenas.append(info)

gdf = gpd.GeoDataFrame(lista_escenas, crs = 'EPSG:4326')
gdf.head()

,id,geometry,gsd,created,sci:doi,datetime,platform,proj:shape,description,instruments,...,landsat:scene_id,landsat:wrs_path,landsat:wrs_type,view:sun_azimuth,landsat:correction,view:sun_elevation,landsat:cloud_cover_land,landsat:collection_number,landsat:collection_category,proj:code
0,LC09_L2SP_026047_20251004_02_T1,"POLYGON ((-99.78727 19.82624, -100.17836 18.09...",30,2025-10-08T09:26:30.550444Z,10.5066/P9OGBGM6,2025-10-04T17:00:17.535650Z,landsat-9,"[7761, 7601]",Landsat Collection 2 Level-2,"[oli, tirs]",...,LC90260472025277LGN00,026,2,136.203191,L2SP,58.620649,51.95,02,T1,EPSG:32614
1,LC09_L2SP_026046_20251004_02_T1,"POLYGON ((-99.46273 21.26923, -99.85249 19.536...",30,2025-10-08T09:26:29.340787Z,10.5066/P9OGBGM6,2025-10-04T16:59:53.636163Z,landsat-9,"[7751, 7601]",Landsat Collection 2 Level-2,"[oli, tirs]",...,LC90260462025277LGN00,026,2,138.120495,L2SP,57.712492,49.11,02,T1,EPSG:32614
2,LC09_L2SP_025047_20250927_02_T1,"POLYGON ((-98.24777 19.82639, -98.63797 18.094...",30,2025-10-02T09:20:23.381761Z,10.5066/P9OGBGM6,2025-09-27T16:54:06.614234Z,landsat-9,"[7711, 7561]",Landsat Collection 2 Level-2,"[oli, tirs]",...,LC90250472025270LGN00,025,2,131.745002,L2SP,60.216227,28.09,02,T1,EPSG:32614
3,LC08_L2SP_026047_20250926_02_T1,"POLYGON ((-99.79474 19.82891, -100.18575 18.09...",30,2025-10-05T09:20:46.826712Z,10.5066/P9OGBGM6,2025-09-26T17:00:14.812341Z,landsat-8,"[7761, 7611]",Landsat Collection 2 Level-2,"[oli, tirs]",...,LC80260472025269LGN00,026,2,131.059268,L2SP,60.424213,41.10,02,T1,EPSG:32614
4,LC08_L2SP_026046_20250926_02_T1,"POLYGON ((-99.46997 21.27165, -99.85965 19.538...",30,2025-10-05T09:20:45.654541Z,10.5066/P9OGBGM6,2025-09-26T16:59:50.917065Z,landsat-8,"[7761, 7601]",Landsat Collection 2 Level-2,"[oli, tirs]",...,LC80260462025269LGN00,026,2,133.246413,L2SP,59.619292,41.72,02,T1,EPSG:32614


In [29]:
# Obtener factores de escala desde los metadatos del item
item = items[68]

green_asset = item.assets['green']

In [40]:
green_band_info = green_asset.extra_fields['raster:bands'][0]
green_scale = green_band_info.get('scale')
green_offset = green_band_info.get('offset')

green_scale, green_offset

(2.75e-05, -0.2)

In [41]:
# Obtener la URL del asset de la banda verde
green_band_url = item.assets["green"].href
green_band_url

'https://landsateuwest.blob.core.windows.net/landsat-c2/level-2/standard/oli-tirs/2025/025/047/LC09_L2SP_025047_20250404_20250405_02_T1/LC09_L2SP_025047_20250404_20250405_02_T1_SR_B3.TIF?st=2025-10-08T18%3A40%3A48Z&se=2025-10-09T19%3A25%3A48Z&sp=rl&sv=2025-07-05&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2025-10-09T15%3A55%3A49Z&ske=2025-10-16T15%3A55%3A49Z&sks=b&skv=2025-07-05&sig=Wdqb51/1sn3hnXcSrjO1uirU560HOuvjTmKAURd7eu4%3D'

# RIOXARRAY

In [ ]:
import rioxarray
import matplotlib.pyplot as plt

https://tutorial.xarray.dev/fundamentals/01_data_structures.html

In [ ]:
# plot la banda


In [ ]:
# Ver su sistema de referencia


In [ ]:
# Obtener los valores como un array de numpy


In [ ]:
# Hacer nan los ceros de mi rioxarray 


In [ ]:
# Reescalar mi rioxarray


In [ ]:
# Ver minimo y maximo de la banda reescalada


In [ ]:
# Exportar la banda reescalada a GeoTIFF


In [ ]:
# Obtener bandas red y nir reescaladas
# Ver minimo y maximo


In [ ]:
# Ver minimo y maximo


In [ ]:
# Historgrama de NDVI


In [ ]:
# Filtrar ndvi a valores valdos 


In [ ]:
# Histograma de NDVI corr


In [ ]:
# Siguiente misión: Usar la variable de '- qa_pixel : Pixel Quality Assessment Band' para hacer el filtro de píxeles anómalos......